In [59]:
from dotenv import load_dotenv

import os

### Set the Environment

In [60]:
load_dotenv()

API_KEY = os.getenv("GEMINI_API_KEY")
PROJECT_ID = os.getenv("PROJECT_ID")

### Call the Gemini-Pro

Vertex AI: https://ai.google.dev/gemini-api/docs/get-started/tutorial?lang=python

LangChain: https://python.langchain.com/v0.1/docs/integrations/chat/google_generative_ai/

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")

In [4]:
prompt = "Explain the quantum mechanic in one sentence"
result = llm.invoke(prompt)

In [5]:
print(result.content)

Quantum mechanics is a branch of science that studies the behavior of matter and energy at the atomic and subatomic levels, where classical mechanics fails to explain their behavior.


### Langchain Schema

In [42]:
# from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI


In [49]:
messages = [
    SystemMessage(content="You are the Physicist and only response the output in Thai language only"),
    HumanMessage(content=prompt)
]

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro", convert_system_message_to_human=True)
result = llm.invoke(messages)

c:\Users\armlo\anaconda3\envs\langchain\Lib\site-packages\langchain_google_genai\chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [50]:
result.content

'กลศาสตร์ควอนตัมอธิบายพฤติกรรมของสสารและพลังงานในระดับอะตอมและอนุภาคมูลฐาน'

### Caching LLM Response

In Memory Caching

In [63]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_google_genai import ChatGoogleGenerativeAI

set_llm_cache(InMemoryCache())

In [64]:
prompt = "Tell me a some dad jokes in two examples"

In [59]:
%%time
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
output = llm.invoke(prompt)
print(output.content)


1. What do you call a fish with no eyes? Fsh!
2. Why did the scarecrow win an award? Because he was outstanding in his field!
3. What do you call a belt made out of watches? A waist of time!
4. Why did the golfer wear two pairs of pants? In case he got a hole-in-one!
5. What do you call a lazy kangaroo? A pouch potato!
CPU times: total: 0 ns
Wall time: 2.24 s


In [60]:
%%time
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
output = llm.invoke(prompt)
print(output.content)


1. What do you call a fish with no eyes? Fsh!
2. Why did the scarecrow win an award? Because he was outstanding in his field!
3. What do you call a belt made out of watches? A waist of time!
4. Why did the golfer wear two pairs of pants? In case he got a hole-in-one!
5. What do you call a lazy kangaroo? A pouch potato!
CPU times: total: 0 ns
Wall time: 999 µs


SQLite Caching

In [65]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="./langchain.db"))

In [66]:
%%time
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
llm.invoke(prompt)

CPU times: total: 15.6 ms
Wall time: 1.5 s


AIMessage(content="1. What do you call a boomerang that doesn't come back? A stick!\n2. Why did the scarecrow win an award? Because he was outstanding in his field!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c82ab426-75c2-4398-b648-25c99ada289a-0', usage_metadata={'input_tokens': 10, 'output_tokens': 39, 'total_tokens': 49})

In [67]:
%%time
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 14.2 ms


AIMessage(content="1. What do you call a boomerang that doesn't come back? A stick!\n2. Why did the scarecrow win an award? Because he was outstanding in his field!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c82ab426-75c2-4398-b648-25c99ada289a-0', usage_metadata={'input_tokens': 10, 'output_tokens': 39, 'total_tokens': 49})

### Langchain Streaming

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
prompt = "Tell me a chorus lyrics of 'talking to the moon' of Bruno mars"

In [13]:
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Talking to the moon
Trying to get to you
In hopes you're on the other side
Talking to me too
Or am I a fool
Who's trying to catch the moon

### Langchain PromptTemplates

In [15]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI


In [22]:
template = """
    You are the expert virologist.
    Write a few sentence to explain about the {virus} in {language} language.
"""

prompt_template = PromptTemplate.from_template(template=template)

In [23]:
prompt = prompt_template.format(virus="HIV", language="Thai")
prompt

'\n    You are the expert virologist.\n    Write a few sentence to explain about the HIV in Thai language.\n'

In [24]:
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
llm.invoke(prompt).content

'HIV (Human Immunodeficiency Virus) เป็นไวรัสที่ทำลายเซลล์เม็ดเลือดขาวชนิดหนึ่ง ทำให้ร่างกายอ่อนแอและติดเชื้อได้ง่าย หากไม่ได้รับการรักษา HIV อาจลุกลามไปเป็นโรคเอดส์ (Acquired Immunodeficiency Syndrome) ซึ่งเป็นระยะสุดท้ายของการติดเชื้อ HIV'

### Langchain ChatPromptTemplates

In [18]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro", convert_system_message_to_human=True)

In [42]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You have to response array in JSON format"),
        HumanMessagePromptTemplate.from_template("Top {n} countries at {continent} order by population")
    ]
)

messages = chat_template.format_messages(n=10, continent="asia")

In [43]:
output = llm.invoke(messages)

c:\Users\armlo\anaconda3\envs\langchain\Lib\site-packages\langchain_google_genai\chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [44]:
print(output.content)

```json
[
  {
    "country": "China",
    "population": 1,444,244,680
  },
  {
    "country": "India",
    "population": 1,393,409,038
  },
  {
    "country": "Indonesia",
    "population": 273,523,615
  },
  {
    "country": "Pakistan",
    "population": 220,892,340
  },
  {
    "country": "Bangladesh",
    "population": 164,689,383
  },
  {
    "country": "Japan",
    "population": 126,440,000
  },
  {
    "country": "Philippines",
    "population": 109,581,078
  },
  {
    "country": "Vietnam",
    "population": 97,338,583
  },
  {
    "country": "Thailand",
    "population": 69,799,924
  },
  {
    "country": "Myanmar",
    "population": 54,409,794
  }
]
```


### Langchain Chained Messages

In [46]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")

In [47]:
template = """
    You are the expert virologist.
    Write a few sentence to explain about the {virus} in {language} language.
"""

prompt_template = PromptTemplate.from_template(template=template)

In [50]:
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True,
)

output = chain.invoke({"virus": "HIV", "language": "Thai"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are the expert virologist.
    Write a few sentence to explain about the HIV in Thai language.


> Finished chain.


In [51]:
output

{'virus': 'HIV',
 'language': 'Thai',
 'text': 'เชื้อ HIV เป็นเชื้อไวรัสที่ทำลายเซลล์เม็ดเลือดขาวชนิด CD4 ซึ่งเป็นเซลล์ที่สำคัญในระบบภูมิคุ้มกันของร่างกาย เมื่อร่างกายมีปริมาณเซลล์ CD4 ต่ำลง ระบบภูมิคุ้มกันก็จะอ่อนแอลง ทำให้เสี่ยงต่อการติดเชื้อฉวยโอกาสหรือโรคมะเร็งได้ง่ายขึ้น'}

### Langchain Squential Chains

In [54]:
# from langchain_google_genai import GoogleGenerativeAI
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain


# llm1 = GoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
# llm2 = GoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")

In [1]:
# prompt_template1 = PromptTemplate.from_template(
#     template = """You are the expert virologist.
#     Write a few sentence to explain about the {virus}"""
# )

# prompt_template2 = PromptTemplate.from_template(
#     template = """Convert the contents to {language}"""
# )

# chain1 = LLMChain(llm=llm1, prompt=prompt_template1.format(virus="HIV"))
# chain2 = LLMChain(llm=llm2, prompt=prompt_template2.format(language="Thai"))

In [2]:
# overall = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
# output = overall.invoke("virus")

### Langchain Agent

Python REPL for Calculator

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [16]:
query = "Calculate the 3rd root of the factorial of 12 and display it with 4 decimal points"
agent_executor.invoke(query)



> Entering new AgentExecutor chain...
Action: Python_REPL
Action Input: print(round(pow(math.factorial(12), 1/3), 4))
Observation: NameError("name 'math' is not defined")
Thought:Action: Python_REPL
Action Input: import math
Observation: 
Thought:Action: Python_REPL
Action Input: print(round(pow(math.factorial(12), 1/3), 4))
Observation: 782.4303

Thought:Final Answer: 782.4303

> Finished chain.


{'input': 'Calculate the 3rd root of the factorial of 12 and display it with 4 decimal points',
 'output': '782.4303'}

DuckDuckGo for Search

In [17]:
# from langchain.tools import DuckDuckGoSearchResults, DuckDuckGoSearchRun, tavily_search
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun, DuckDuckGoSearchResults

In [18]:
search = DuckDuckGoSearchRun()
search_results = DuckDuckGoSearchResults()

In [19]:
response = search.run("What is Git?")
print(response)

Git is a version control system that tracks file changes and GitHub is a platform that allows developers to collaborate and store their code in the cloud. Think of it this way: Git is responsible for everything GitHub-related that happens locally on your computer. They both work together to make building, scaling, securing, and storing software ... What is Git? Git is a distributed version control system designed to handle everything from small to very large projects with speed and efficiency. It was created by Linus Torvalds in 2005 for the development of the Linux kernel. In this article, we will discuss about Git, its features, advantages, and disadvantages. Learn what Git is, how to install and configure it, and how to use its basic commands to create, commit, and sync repositories. This guide covers the essential concepts and workflows of Git for developers. Git offers a variety of configurations that can streamline your workflow. In this section, I will guide you through the proc

In [25]:
response = search_results.run("What is Git?")
print(response)

[snippet: Git is a version control system that tracks file changes and GitHub is a platform that allows developers to collaborate and store their code in the cloud. Think of it this way: Git is responsible for everything GitHub-related that happens locally on your computer. They both work together to make building, scaling, securing, and storing software ..., title: What is Git? Our beginner's guide to version control, link: https://github.blog/2024-05-27-what-is-git-our-beginners-guide-to-version-control/], [snippet: What is Git? Git is a distributed version control system designed to handle everything from small to very large projects with speed and efficiency. It was created by Linus Torvalds in 2005 for the development of the Linux kernel. In this article, we will discuss about Git, its features, advantages, and disadvantages., title: What is Git? - GeeksforGeeks, link: https://www.geeksforgeeks.org/what-is-git/], [snippet: Learn what Git is, how to install and configure it, and ho

In [38]:
###################################### Convert to string to JSON #######################################

# import re
# import json

# # String pattern to be converted
# # Function to convert string pattern to JSON format
# def convert_to_json(data_string: str) -> json:
#     # Regular expression to extract snippets, titles, and links
#     pattern = re.compile(r'\[snippet: (.*?), title: (.*?), link: (.*?)\]')
    
#     # Finding all matches in the string
#     matches = pattern.findall(data_string)
    
#     # Preparing the list of dictionaries
#     json_list = []
#     for match in matches:
#         snippet, title, link = match
#         entry_dict = {
#             "snippet": snippet.strip(),
#             "title": title.strip(),
#             "link": link.strip()
#         }
#         json_list.append(entry_dict)
    
#     # Convert the list to JSON format
#     json_output = json.dumps(json_list, indent=4)
#     return json.loads(json_output)


Tavily AI for Search

In [39]:
import os
from langchain_community.tools.tavily_search.tool import TavilyAnswer, TavilySearchResults

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [42]:
search = TavilyAnswer()

In [43]:
response = search.run("What is Git?")
print(response)

Git is a distributed version control system that handles data as a stream of snapshots, making it distinct from other version control systems. It is known for its performance, security, flexibility, and support for branching, tagging, and nonlinear development workflows. As of the latest data available, Git is the most widely used source-code management tool, with significant adoption among professional software developers and open-source projects. It is favored by developers and has various extensions like Git LFS.


In [44]:
search_results = TavilySearchResults()

In [46]:
response = search_results.run("What is Git?")

In [54]:
print(response[0])

{'url': 'https://en.wikipedia.org/wiki/Git', 'content': "SourceForge, Bitbucket and GitLab.[92][17][18][19][20]\nAdoption[edit]\nThe Eclipse Foundation reported in its annual community survey that as of May 2014, Git is now the most widely used source-code management tool, with 42.9% of professional software developers reporting that they use Git as their primary source-control system[93] compared with 36.3% in 2013, 32% in 2012; or for Git responses excluding use of GitHub: 33.3% in 2014, 30.3% in 2013, 27.6% in 2012 and 12.8% in 2011.[94] Open-source directory Black Duck Open Hub reports a similar uptake among open-source projects.[95]\nStack Overflow has included version control in their annual developer survey[96] in 2015 (16,694 responses),[97] 2017 (30,730 responses),[98] 2018 (74,298 responses)[99] and 2022 (71,379 responses).[15] Git was the overwhelming favorite of responding developers in these surveys, reporting as high as 93.9% in 2022.\n It has bindings for many programmin

### Langchain Owned ReAct Agent

In [61]:
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_experimental.tools.python.tool import PythonREPLTool


In [63]:
llm = GoogleGenerativeAI(
    google_api_key=API_KEY,
    temperature=0,
    model="gemini-1.5-pro"
)

In [65]:
template = """
Answer the following question as best you can. 
Questions: {question} 
Answer: 
"""

prompt_template = PromptTemplate.from_template(template=template)
prompt = hub.pull("hwchase17/react")